<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
# This cell is tagged "parameters" to enable running this notebook from the command line with papermill
# Example usage:
# papermill -p USER aubreymoore -p USER_FULL_NAME 'Aubrey Moore' inat_labels.ipynb aubreymoore.ipynb

FONT_SIZE = 3.5    # points
FONT = 'helvetica'
YMIN = 1           # inches
LABEL_WIDTH = 1.0  # inches
LABEL_HEIGHT = 1.0 # inches
XMIN = 0.5         # inches

NROWS = 9
NCOLS = 7

USER = 'aubreymoore'
USER_FULL_NAME = 'Aubrey Moore'

DESC_MAX_CHARS = 100

In [2]:
# Parameters
USER = "stewjohn"
USER_FULL_NAME = "Stewart Johnny"


# TO DO
* papermill
* add barcode
* check on determined by
* document code
* add page title
* add description field

In [3]:
import subprocess
import sqlite3
from fpdf import FPDF
import qrcode
from datetime import datetime

In [4]:
def add_taxonomy(obs_dict):
    ancestry = obs_dict['ancestry'].split('/')
    ancestry.append(obs_dict['taxon'])
    for t in ancestry:
        sql = f'SELECT rank, name FROM taxons WHERE id == {t}'
        cursor = conn.execute(sql)
        for row in cursor:
            obs_dict[row[0]] = row[1]

# obs_dict = {'observed_on': '2021-08-25',
#  'taxon': 145541,
#  'species_guess': None,
#  'longitude': '144.8007189136',
#  'latitude': '13.4301838323',
#  'project_ids': '[1627]',
#  'ancestry': '48460/1/47120/372739/47158/184884/47822/48091/372853/414448/49995/326181/364082/520020'}
# add_taxonomy(mydict)
# obs_dict

In [5]:
def get_label_data():
    data_list = []
    sql = """
    SELECT observations.id, name, rank, observed_on, taxon, place_guess, longitude, latitude, description, project_ids, taxons.ancestry
    FROM observations, taxons
    WHERE 
       taxons.id == taxon
       AND project_ids LIKE '%1627%'
       AND observed_on > '2021-08-01';
    """
    cursor = conn.execute(sql)
    for r in cursor:
        obs_dict = dict(r)
        add_taxonomy(obs_dict)
        for key in ['ancestry', 'rank', 'name', 'project_ids', 'taxon']:
            del obs_dict[key]
        data_list.append(obs_dict)        
    return data_list

# get_label_data()

In [6]:
class PDF(FPDF):
    
    def add_label(self, x, y, obs):
        self.set_xy(x, y)
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        #print(f'1:{self.get_x()} {self.get_y()}')
        self.multi_cell(txt=obs['place_guess'], w=LABEL_WIDTH, ln=2, align='L')
        #print(f'2:{self.get_x()} {self.get_y()}')
        
        lat = float(obs['latitude'])
        lat = round(lat,6)
        if (lat > 0):
            lat_suffix = '°N'
        else:
            lat_suffix = '°S'       
        
        lon = float(obs['longitude'])
        lon = round(lon, 6)
        if (lon > 0):
            lon_suffix = '°E'
        else:
            lon_suffix = '°W'
        txt = f'{lat}{lat_suffix} {lon}{lon_suffix}' 
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')

        txt = obs['description']
        if len(txt) > 1:
            if len(txt) > DESC_MAX_CHARS:
                txt = f'{txt[0:DESC_MAX_CHARS]}...'
            self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
       
        txt = f'Col. {USER_FULL_NAME} {obs["observed_on"]}'
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
        
        self.multi_cell(txt='---', w=LABEL_WIDTH, ln=2, align='L')
        
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        
        if obs.get('order'):
            txt = obs['order'].upper()
            self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
            
        if obs.get('family'):
            txt = f'   {obs["family"].upper()}'
            self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
            
        if obs.get('species'):   # species binomial available
            txt = f'     {obs["species"]}'
            self.set_font(style="I")
            self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
            self.set_font(style="")
        else:
            if obs.get('genus'):   # only genus available
                txt = f'     {obs["genus"]}'
                self.set_font(style="I")
                self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
                self.set_font(style="")

        # Check on this.
        txt = f'Det. {USER_FULL_NAME} {obs["observed_on"]}'
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
        
        # barcode
        
        self.multi_cell(txt='---', w=LABEL_WIDTH, ln=2, align='L')
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        url = "https://www.inaturalist.org/observations/{obs['id']}"
        img = qrcode.make(url)
        self.image(img.get_image(), w=0.25)

In [7]:
def generate_labels():

    pdf = PDF(orientation="P", unit="in", format="Letter")
    pdf.set_font(family=FONT, size=FONT_SIZE)
    
    n = len(data_list)
    i = 0
    pages = 1 + n // (NROWS * NCOLS)
    for p in range(pages):
        print(f'Adding page {p}')
        pdf.add_page()
        txt=f'{USER_FULL_NAME}          Page {p+1} of {pages}          {datetime.now()}'
        pdf.set_font(size=14)
        pdf.multi_cell(txt=txt, w = 6)
        pdf.set_font(size=FONT_SIZE)
        r = 0
        c = 0
        for r in range(NROWS):
            y = YMIN + r * LABEL_HEIGHT
            for c in range(NCOLS):
                x = XMIN + c * LABEL_WIDTH
#                 print(f'{i} {p} {r} {c} {x} {y}')
#                 print(data_list[i])
                pdf.add_label(x, y, data_list[i])
                i += 1
                if i == n:
                    print(f"writing {pdf_filename}")
                    pdf.output(pdf_filename)
                    return
                
#generate_labels()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
# MAIN

# Download USER observations from the iNaturalist Insects of Micronesia Project as an SQLite database

print('Downloading data from iNaturalist. This may take a few minutes.')
db_filename = f"{USER_FULL_NAME.replace(' ', '_')}.db"
result = subprocess.run(['inaturalist-to-sqlite', db_filename, USER]) 
print(result)

# Open the database and extract label data

conn = sqlite3.connect(db_filename)
conn.row_factory = sqlite3.Row
data_list = get_label_data()

# Generate labels

pdf_filename = db_filename.replace('.db', '.pdf')
generate_labels()

print('FINISHED')

CompletedProcess(args=['inaturalist-to-sqlite', 'Stewart_Johnny.db', 'stewjohn'], returncode=0)
Adding page 0


TypeError: object of type 'NoneType' has no len()